In [1]:
!pwd
!ls

/home/jupyter
SQuaD.ipynb
XLM-Roberta
bert_bert-base-multilingual-cased_squad-2_batch_large
bert_bert-base-multilingual-cased_train_tydiqa-train-formatted_batch_large
checkpoint-35620-epoch-5
checkpoint-7124-epoch-1
download_from_bucket.ipynb
test_data-pt.json
train-v2.0-pt.json
train_data-pt.json
training_progress_scores.csv
tutorials
upload_to_bucket.ipynb
xlmroberta_xlm-roberta-base_train_tydiqa-train-formatted_batch_large
xlmroberta_xlm-roberta-large_train_tydiqa-train-formatted_batch_large


In [2]:
import os
import json

# !pip install simpletransformers
# !pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
from simpletransformers.question_answering import QuestionAnsweringModel

# Split data

In [ ]:
# """ Split train/test into files """
# with open('train-v2.0-pt.json', 'r') as f:
#     data = json.load(f)

# split_ratio = 0.15
# data = [item for topic in data['data'] for item in topic['paragraphs'] ]
# test_size = int(split_ratio * len(data))
# test_data = data[:test_size]
# train_data = data[test_size:]

# with open('train_data-pt.json', 'w') as f:
#     json.dump(train_data, f, indent=2)

# with open('test_data-pt.json', 'w') as f:
#     json.dump(test_data, f, indent=2)

# Load Model

In [3]:
# train_args = {
#     'learning_rate': 3e-5,
#     'num_train_epochs': 2,
#     'max_seq_length': 512,
#     'doc_stride': 384,
#     'overwrite_output_dir': True,
#     'reprocess_input_data': False,
#     'train_batch_size': 4,
#     'gradient_accumulation_steps': 4,

#     'n_best_size': 3,
#     'eval_batch_size': 2,
#     "evaluate_during_training": True,
#     'evaluate_during_training_verbose': True,
#     "use_cached_eval_features": True,
# }

train_args = {
  "num_train_epochs": 2,
  "max_seq_lenght": 384,
  "train_batch_size": 16,
  "gradient_accumulation_steps": 1,
  "eval_batch_size": 16,
  "save_steps": -1,
  "save_model_every_epoch": True,
  "evaluate_during_training_steps": -1,
  "evaluate_during_training": True,
  "evaluate_during_training_verbose": True,
  "use_cached_eval_features": True
}

model_dir = 'xlmroberta_xlm-roberta-large_train_tydiqa-train-formatted_batch_large/best_model'
# model_dir = 'outputs/checkpoint-16842-epoch-1'
# model_dir = 'neuralmind/bert-large-portuguese-cased' # <-- BERT-base model
# model_dir = 'salti/bert-base-multilingual-cased-finetuned-squad'
# model_dir = 'XLM-Roberta/checkpoint-5560-epoch-2'

# model_type = 'bert'
model_type = 'xlmroberta'
print('loading model')
model = QuestionAnsweringModel(model_type, model_dir, args=train_args, use_cuda=True)
print('model loaded')

loading model
model loaded


# Train Model

In [ ]:
train_dataset_path = 'train_data-pt.json'
eval_dataset_path = 'test_data-pt.json'

with open(train_dataset_path, 'r') as f:
    train_data = json.load(f)

with open(eval_dataset_path, 'r') as f:
    eval_data = json.load(f)

model.train_model(train_data[:10], eval_data=eval_data[:10], output_dir='train_eval')

add example index and unique id: 100%|██████████| 71/71 [00:00<00:00, 428482.85it/s]


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)

convert squad examples to features: 100%|██████████| 125/125 [00:01<00:00, 99.89it/s]A

add example index and unique id: 100%|██████████| 125/125 [00:00<00:00, 259291.79it/s]



convert squad examples to features: 100%|██████████| 125/125 [00:01<00:00, 98.89it/s]A

add example index and unique id: 100%|██████████| 125/125 [00:00<00:00, 431512.76it/s]


# Exemplo

In [13]:
word_splits = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000]

def get_contexts(context):
    contexts = []
    context_words_list = context.split(' ')
    print(len(context_words_list))
    for split in word_splits:
        if len(context_words_list) > split:
            split_context = ' '.join(context_words_list[:split])
            contexts.append(split_context)
    return contexts

context = (
  "A forma de pensar e se relacionar da sociedade mudou. E o surgimento de novas tecnologias foi o maior influenciador desta mudança, que afetou sobretudo o jeito de se comunicar das empresas e os objetivos da Ilhasoft. "
  "A Plataforma Push, produto Ilhasoft, tem participado ativamente desta mudança com soluções que tornaram o relacionamento entre empresas e clientes mais dinâmico e positivo."
  "Unindo a vontade de causar impacto social com o propósito de assegurar a melhor experiência para o usuário, a Ilhasoft utiliza a inteligência artificial para solucionar problemas de comunicação. "
  "Neste artigo, vamos conhecer algumas das soluções oferecidas pela Ilhasoft e entender como elas podem auxiliar as estratégias de comunicação do seu negócio. Confira! "
  "Um dos maiores desafios das empresas é otimizar o seu serviço de atendimento ao cliente. "
  "Alguns negócios recebem um alto volume de solicitações do seu público, seja para tirar dúvidas, solicitar suporte ou resolver um problema específico. "
  "A alta quantidade de solicitações de contato muitas vezes demanda muitos atendentes. Isso costuma ser inviável, devido ao alto custo envolvido na contratação de colaboradores. O resultado muitas vezes é uma equipe de SAC com sobrecarga de trabalho. "
  "A saída economicamente viável para este tipo de problema está na tecnologia, mais especificamente nos chatbots. Com eles, é possível automatizar a comunicação com o seu público e assim desafogar a equipe humana de atendentes. "
  "Push Platform é a solução em chatbots da Ilhasoft. Com ela, é possível criar automações para agilizar tarefas monótonas e repetitivas, programando um assistente virtual para responder dúvidas frequentes. Além disso, ele também pode ser capaz de enviar materiais e transmitir informações para o usuário, por exemplo. "
  "Desta forma, o chatbot qualifica a conversa com o usuário e o direciona à sua equipe humana caso necessário. Assim, boa parte do atendimento é automatizado e a sua equipe humana tem mais tempo para focar em problemas complexos e se dedicar a ações estratégicas. "
  "Como dissemos no início deste artigo, a tecnologia mudou a forma como pessoas e empresas se comunicam. "
  "Hoje, o público espera um atendimento omnichannel, ou seja, multicanal. O usuário espera encontrar o mesmo produto ou serviço, com os mesmos preços, condições e atendimento em cada um dos canais que a sua empresa atua. "
  "Para isso, é preciso integrar todos os processos do seu negócio para oferecer ao usuário uma experiência positiva e única independente do canal utilizado por ele para se comunicar com a sua empresa. "
  "Push Platform atende muito bem a este papel, ao disponibilizar a criação de chatbots omnichannel. "
  "Eles possuem integração com diversos canais de comunicação, como Whatsapp, Telegram, Facebook Messenger, seu site e até o aplicativo da sua empresa. Conectando os assistentes a um sistema de livechat, a estratégia omnichannel atinge outros níveis de produtividade nas empresas! "
  "Ao utilizar a estratégia de chatbots no seu atendimento omnichannel, é possível captar dados do consumidor em cada um dos canais em que ele interage. Assim, é fácil traçar um perfil que pode ser utilizado em estratégias de marketing e vendas. "
  "Nada pior do que iniciar uma conversa com o chatbot de uma empresa e perceber que ele soa robótico. Pior ainda é ele não conseguir interpretar e solucionar as suas solicitações. "
  "Quando isso acontece, a experiência do usuário acaba sendo negativa. Isso pode prejudicar a sua marca e fazer com que a sua empresa perca oportunidades de negócio. "
  "Para evitar chatbots engessados, a Ilhasoft desenvolveu o Bothub. Esse sistema de Processamento de Linguagem Natural (PLN) utiliza Inteligência Artificial (IA) para ajudar os chatbots a compreenderem a linguagem que nós falamos. "
  "O mais interessante é que o Bothub é uma plataforma colaborativa, totalmente aberta e democrática. Qualquer pessoa pode se cadastrar gratuitamente, criar sua própria IA e compartilhar os dados com a comunidade. Também é possível contribuir com inteligências já existentes adicionando mais exemplos ou suporte a outros idiomas. "
  "Assim, com o uso do Push Platform e do Bothub, duas soluções Ilhasoft, é possível criar chatbots inteligentes, capazes de interpretar e entregar a melhor solução para o seu usuário, através de uma conversa inteligente e natural. "
  "Vale ressaltar que o Bothub ganhou o 2º lugar da categoria “Desenvolvedores de Apps e Conteúdo” no Anuário Tele.Síntese de Inovação em Comunicações em 2018. "
  "Não há dúvidas sobre a forte presença do WhatsApp nas estratégias de comunicação das empresas. Para ter ideia, 76% dos usuários do WhatsApp afirmam utilizar o aplicativo para se comunicar com marcas, de acordo com pesquisa realizada pela Mobile Time. "
  "E isso não é surpresa, afinal o WhatsApp tem investido em soluções voltadas para o uso corporativo do aplicativo. "
  "Um exemplo é lançamento da versão Business e a liberação da sua API em 2018. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
    "A forma de pensar e se relacionar da sociedade mudou. E o surgimento de novas tecnologias foi o maior influenciador desta mudança, que afetou sobretudo o jeito de se comunicar das empresas e os objetivos da Ilhasoft. "
  "A Plataforma Push, produto Ilhasoft, tem participado ativamente desta mudança com soluções que tornaram o relacionamento entre empresas e clientes mais dinâmico e positivo."
  "Unindo a vontade de causar impacto social com o propósito de assegurar a melhor experiência para o usuário, a Ilhasoft utiliza a inteligência artificial para solucionar problemas de comunicação. "
  "Neste artigo, vamos conhecer algumas das soluções oferecidas pela Ilhasoft e entender como elas podem auxiliar as estratégias de comunicação do seu negócio. Confira! "
  "Um dos maiores desafios das empresas é otimizar o seu serviço de atendimento ao cliente. "
  "Alguns negócios recebem um alto volume de solicitações do seu público, seja para tirar dúvidas, solicitar suporte ou resolver um problema específico. "
  "A alta quantidade de solicitações de contato muitas vezes demanda muitos atendentes. Isso costuma ser inviável, devido ao alto custo envolvido na contratação de colaboradores. O resultado muitas vezes é uma equipe de SAC com sobrecarga de trabalho. "
  "A saída economicamente viável para este tipo de problema está na tecnologia, mais especificamente nos chatbots. Com eles, é possível automatizar a comunicação com o seu público e assim desafogar a equipe humana de atendentes. "
  "Push Platform é a solução em chatbots da Ilhasoft. Com ela, é possível criar automações para agilizar tarefas monótonas e repetitivas, programando um assistente virtual para responder dúvidas frequentes. Além disso, ele também pode ser capaz de enviar materiais e transmitir informações para o usuário, por exemplo. "
  "Desta forma, o chatbot qualifica a conversa com o usuário e o direciona à sua equipe humana caso necessário. Assim, boa parte do atendimento é automatizado e a sua equipe humana tem mais tempo para focar em problemas complexos e se dedicar a ações estratégicas. "
  "Como dissemos no início deste artigo, a tecnologia mudou a forma como pessoas e empresas se comunicam. "
  "Hoje, o público espera um atendimento omnichannel, ou seja, multicanal. O usuário espera encontrar o mesmo produto ou serviço, com os mesmos preços, condições e atendimento em cada um dos canais que a sua empresa atua. "
  "Para isso, é preciso integrar todos os processos do seu negócio para oferecer ao usuário uma experiência positiva e única independente do canal utilizado por ele para se comunicar com a sua empresa. "
  "Push Platform atende muito bem a este papel, ao disponibilizar a criação de chatbots omnichannel. "
  "Eles possuem integração com diversos canais de comunicação, como Whatsapp, Telegram, Facebook Messenger, seu site e até o aplicativo da sua empresa. Conectando os assistentes a um sistema de livechat, a estratégia omnichannel atinge outros níveis de produtividade nas empresas! "
  "Ao utilizar a estratégia de chatbots no seu atendimento omnichannel, é possível captar dados do consumidor em cada um dos canais em que ele interage. Assim, é fácil traçar um perfil que pode ser utilizado em estratégias de marketing e vendas. "
  "Nada pior do que iniciar uma conversa com o chatbot de uma empresa e perceber que ele soa robótico. Pior ainda é ele não conseguir interpretar e solucionar as suas solicitações. "
  "Quando isso acontece, a experiência do usuário acaba sendo negativa. Isso pode prejudicar a sua marca e fazer com que a sua empresa perca oportunidades de negócio. "
  "Para evitar chatbots engessados, a Ilhasoft desenvolveu o Bothub. Esse sistema de Processamento de Linguagem Natural (PLN) utiliza Inteligência Artificial (IA) para ajudar os chatbots a compreenderem a linguagem que nós falamos. "
  "O mais interessante é que o Bothub é uma plataforma colaborativa, totalmente aberta e democrática. Qualquer pessoa pode se cadastrar gratuitamente, criar sua própria IA e compartilhar os dados com a comunidade. Também é possível contribuir com inteligências já existentes adicionando mais exemplos ou suporte a outros idiomas. "
  "Assim, com o uso do Push Platform e do Bothub, duas soluções Ilhasoft, é possível criar chatbots inteligentes, capazes de interpretar e entregar a melhor solução para o seu usuário, através de uma conversa inteligente e natural. "
  "Vale ressaltar que o Bothub ganhou o 2º lugar da categoria “Desenvolvedores de Apps e Conteúdo” no Anuário Tele.Síntese de Inovação em Comunicações em 2018. "
  "Não há dúvidas sobre a forte presença do WhatsApp nas estratégias de comunicação das empresas. Para ter ideia, 76% dos usuários do WhatsApp afirmam utilizar o aplicativo para se comunicar com marcas, de acordo com pesquisa realizada pela Mobile Time. "
  "E isso não é surpresa, afinal o WhatsApp tem investido em soluções voltadas para o uso corporativo do aplicativo. "
  "Um exemplo é lançamento da versão Business e a liberação da sua API em 2018. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
    "A forma de pensar e se relacionar da sociedade mudou. E o surgimento de novas tecnologias foi o maior influenciador desta mudança, que afetou sobretudo o jeito de se comunicar das empresas e os objetivos da Ilhasoft. "
  "A Plataforma Push, produto Ilhasoft, tem participado ativamente desta mudança com soluções que tornaram o relacionamento entre empresas e clientes mais dinâmico e positivo."
  "Unindo a vontade de causar impacto social com o propósito de assegurar a melhor experiência para o usuário, a Ilhasoft utiliza a inteligência artificial para solucionar problemas de comunicação. "
  "Neste artigo, vamos conhecer algumas das soluções oferecidas pela Ilhasoft e entender como elas podem auxiliar as estratégias de comunicação do seu negócio. Confira! "
  "Um dos maiores desafios das empresas é otimizar o seu serviço de atendimento ao cliente. "
  "Alguns negócios recebem um alto volume de solicitações do seu público, seja para tirar dúvidas, solicitar suporte ou resolver um problema específico. "
  "A alta quantidade de solicitações de contato muitas vezes demanda muitos atendentes. Isso costuma ser inviável, devido ao alto custo envolvido na contratação de colaboradores. O resultado muitas vezes é uma equipe de SAC com sobrecarga de trabalho. "
  "A saída economicamente viável para este tipo de problema está na tecnologia, mais especificamente nos chatbots. Com eles, é possível automatizar a comunicação com o seu público e assim desafogar a equipe humana de atendentes. "
  "Push Platform é a solução em chatbots da Ilhasoft. Com ela, é possível criar automações para agilizar tarefas monótonas e repetitivas, programando um assistente virtual para responder dúvidas frequentes. Além disso, ele também pode ser capaz de enviar materiais e transmitir informações para o usuário, por exemplo. "
  "Desta forma, o chatbot qualifica a conversa com o usuário e o direciona à sua equipe humana caso necessário. Assim, boa parte do atendimento é automatizado e a sua equipe humana tem mais tempo para focar em problemas complexos e se dedicar a ações estratégicas. "
  "Como dissemos no início deste artigo, a tecnologia mudou a forma como pessoas e empresas se comunicam. "
  "Hoje, o público espera um atendimento omnichannel, ou seja, multicanal. O usuário espera encontrar o mesmo produto ou serviço, com os mesmos preços, condições e atendimento em cada um dos canais que a sua empresa atua. "
  "Para isso, é preciso integrar todos os processos do seu negócio para oferecer ao usuário uma experiência positiva e única independente do canal utilizado por ele para se comunicar com a sua empresa. "
  "Push Platform atende muito bem a este papel, ao disponibilizar a criação de chatbots omnichannel. "
  "Eles possuem integração com diversos canais de comunicação, como Whatsapp, Telegram, Facebook Messenger, seu site e até o aplicativo da sua empresa. Conectando os assistentes a um sistema de livechat, a estratégia omnichannel atinge outros níveis de produtividade nas empresas! "
  "Ao utilizar a estratégia de chatbots no seu atendimento omnichannel, é possível captar dados do consumidor em cada um dos canais em que ele interage. Assim, é fácil traçar um perfil que pode ser utilizado em estratégias de marketing e vendas. "
  "Nada pior do que iniciar uma conversa com o chatbot de uma empresa e perceber que ele soa robótico. Pior ainda é ele não conseguir interpretar e solucionar as suas solicitações. "
  "Quando isso acontece, a experiência do usuário acaba sendo negativa. Isso pode prejudicar a sua marca e fazer com que a sua empresa perca oportunidades de negócio. "
  "Para evitar chatbots engessados, a Ilhasoft desenvolveu o Bothub. Esse sistema de Processamento de Linguagem Natural (PLN) utiliza Inteligência Artificial (IA) para ajudar os chatbots a compreenderem a linguagem que nós falamos. "
  "O mais interessante é que o Bothub é uma plataforma colaborativa, totalmente aberta e democrática. Qualquer pessoa pode se cadastrar gratuitamente, criar sua própria IA e compartilhar os dados com a comunidade. Também é possível contribuir com inteligências já existentes adicionando mais exemplos ou suporte a outros idiomas. "
  "Assim, com o uso do Push Platform e do Bothub, duas soluções Ilhasoft, é possível criar chatbots inteligentes, capazes de interpretar e entregar a melhor solução para o seu usuário, através de uma conversa inteligente e natural. "
  "Vale ressaltar que o Bothub ganhou o 2º lugar da categoria “Desenvolvedores de Apps e Conteúdo” no Anuário Tele.Síntese de Inovação em Comunicações em 2018. "
  "Não há dúvidas sobre a forte presença do WhatsApp nas estratégias de comunicação das empresas. Para ter ideia, 76% dos usuários do WhatsApp afirmam utilizar o aplicativo para se comunicar com marcas, de acordo com pesquisa realizada pela Mobile Time. "
  "E isso não é surpresa, afinal o WhatsApp tem investido em soluções voltadas para o uso corporativo do aplicativo. "
  "Um exemplo é lançamento da versão Business e a liberação da sua API em 2018. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
)

import time
import statistics

questions = ["Quem e o CEO e cofundador do facebook?", "Oque e o Bothub?", "Qual minha comida favorita?", "Qual um dos maiores desafios da empresa?", "Oque e o Push Whatsapp business api?"]
contexts = get_contexts(context)

n_predicts = 3
parse_times = []
for context in contexts:
    for question in questions:
        query = [
            {
              'context': context,
              'qas': [
                  {'id': '0', 'question': question},
              ]
            }
        ]
        question_time_start = time.time()
        for _ in range(n_predicts):
            
            pred = model.predict(query)
            print("Pergunta: ", question)
            print("Resposta: ", pred[0][0]['answer'][0])
            print("Confiança: ", pred[1][0]['probability'][0])
            print()
        parse_times.append(round((time.time() - question_time_start)/n_predicts, 3))

print(parse_times)
print(f"mean: {statistics.mean(parse_times)}")
print(f"median: {statistics.median(parse_times)}")
print(f"min: {min(parse_times)}")
print(f"max: {max(parse_times)}")

y_axis = []
for i in range(len(word_splits)):
    print(word_splits[i])
    offset = i * len(questions)
    word_splits_times = parse_times[offset:offset + len(questions)]
    y_axis.append(statistics.mean(word_splits_times))
    
print(f"x: {word_splits}")
print(f"y: {y_axis}")

%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd


fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.plot(word_splits, new_y_axis)
ax.set_xlabel('words')
ax.set_ylabel('seconds')

3775


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11428.62it/s]



Pergunta:  Quem e o CEO e cofundador do facebook?
Resposta:  
Confiança:  0.997421503544368



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 14513.16it/s]



Pergunta:  Quem e o CEO e cofundador do facebook?
Resposta:  
Confiança:  0.997421503544368



...


Pergunta:  Oque e o Bothub?
Resposta:  
Confiança:  0.9911564107937015



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 17549.39it/s]



Pergunta:  Qual minha comida favorita?
Resposta:  
Confiança:  0.9999999709341025



Process ForkPoolWorker-929:
Process ForkPoolWorker-930:
Exception ignored in: <function _after_fork at 0x7ff4bb41ba70>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 1375, in _after_fork
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/conda/lib/python3.7/multiprocessing/proces

KeyboardInterrupt: 

# example

In [7]:
context = (
  "A forma de pensar e se relacionar da sociedade mudou. E o surgimento de novas tecnologias foi o maior influenciador desta mudança, que afetou sobretudo o jeito de se comunicar das empresas e os objetivos da Ilhasoft. "
  "A Plataforma Push, produto Ilhasoft, tem participado ativamente desta mudança com soluções que tornaram o relacionamento entre empresas e clientes mais dinâmico e positivo."
  "Unindo a vontade de causar impacto social com o propósito de assegurar a melhor experiência para o usuário, a Ilhasoft utiliza a inteligência artificial para solucionar problemas de comunicação. "
  "Neste artigo, vamos conhecer algumas das soluções oferecidas pela Ilhasoft e entender como elas podem auxiliar as estratégias de comunicação do seu negócio. Confira! "
  "Um dos maiores desafios das empresas é otimizar o seu serviço de atendimento ao cliente. "
  "Alguns negócios recebem um alto volume de solicitações do seu público, seja para tirar dúvidas, solicitar suporte ou resolver um problema específico. "
  "A alta quantidade de solicitações de contato muitas vezes demanda muitos atendentes. Isso costuma ser inviável, devido ao alto custo envolvido na contratação de colaboradores. O resultado muitas vezes é uma equipe de SAC com sobrecarga de trabalho. "
  "A saída economicamente viável para este tipo de problema está na tecnologia, mais especificamente nos chatbots. Com eles, é possível automatizar a comunicação com o seu público e assim desafogar a equipe humana de atendentes. "
  "Push Platform é a solução em chatbots da Ilhasoft. Com ela, é possível criar automações para agilizar tarefas monótonas e repetitivas, programando um assistente virtual para responder dúvidas frequentes. Além disso, ele também pode ser capaz de enviar materiais e transmitir informações para o usuário, por exemplo. "
  "Desta forma, o chatbot qualifica a conversa com o usuário e o direciona à sua equipe humana caso necessário. Assim, boa parte do atendimento é automatizado e a sua equipe humana tem mais tempo para focar em problemas complexos e se dedicar a ações estratégicas. "
  "Como dissemos no início deste artigo, a tecnologia mudou a forma como pessoas e empresas se comunicam. "
  "Hoje, o público espera um atendimento omnichannel, ou seja, multicanal. O usuário espera encontrar o mesmo produto ou serviço, com os mesmos preços, condições e atendimento em cada um dos canais que a sua empresa atua. "
  "Para isso, é preciso integrar todos os processos do seu negócio para oferecer ao usuário uma experiência positiva e única independente do canal utilizado por ele para se comunicar com a sua empresa. "
  "Push Platform atende muito bem a este papel, ao disponibilizar a criação de chatbots omnichannel. "
  "Eles possuem integração com diversos canais de comunicação, como Whatsapp, Telegram, Facebook Messenger, seu site e até o aplicativo da sua empresa. Conectando os assistentes a um sistema de livechat, a estratégia omnichannel atinge outros níveis de produtividade nas empresas! "
  "Ao utilizar a estratégia de chatbots no seu atendimento omnichannel, é possível captar dados do consumidor em cada um dos canais em que ele interage. Assim, é fácil traçar um perfil que pode ser utilizado em estratégias de marketing e vendas. "
  "Nada pior do que iniciar uma conversa com o chatbot de uma empresa e perceber que ele soa robótico. Pior ainda é ele não conseguir interpretar e solucionar as suas solicitações. "
  "Quando isso acontece, a experiência do usuário acaba sendo negativa. Isso pode prejudicar a sua marca e fazer com que a sua empresa perca oportunidades de negócio. "
  "Para evitar chatbots engessados, a Ilhasoft desenvolveu o Bothub. Esse sistema de Processamento de Linguagem Natural (PLN) utiliza Inteligência Artificial (IA) para ajudar os chatbots a compreenderem a linguagem que nós falamos. "
  "O mais interessante é que o Bothub é uma plataforma colaborativa, totalmente aberta e democrática. Qualquer pessoa pode se cadastrar gratuitamente, criar sua própria IA e compartilhar os dados com a comunidade. Também é possível contribuir com inteligências já existentes adicionando mais exemplos ou suporte a outros idiomas. "
  "Assim, com o uso do Push Platform e do Bothub, duas soluções Ilhasoft, é possível criar chatbots inteligentes, capazes de interpretar e entregar a melhor solução para o seu usuário, através de uma conversa inteligente e natural. "
  "Vale ressaltar que o Bothub ganhou o 2º lugar da categoria “Desenvolvedores de Apps e Conteúdo” no Anuário Tele.Síntese de Inovação em Comunicações em 2018. "
  "Não há dúvidas sobre a forte presença do WhatsApp nas estratégias de comunicação das empresas. Para ter ideia, 76% dos usuários do WhatsApp afirmam utilizar o aplicativo para se comunicar com marcas, de acordo com pesquisa realizada pela Mobile Time. "
  "E isso não é surpresa, afinal o WhatsApp tem investido em soluções voltadas para o uso corporativo do aplicativo. "
  "Um exemplo é lançamento da versão Business e a liberação da sua API em 2018. "
  "E a tendência é que o aplicativo continue investindo em recursos para empresas. O CEO e cofundador do Facebook, Mark Zuckerberg, confirmou que pretende lançar oficialmente o WhatsApp Payments (solução para pagamentos dentro do app) ainda este ano. "
  "De olho na evolução e importância deste canal de comunicação para as empresas, a Ilhasoft firmou uma parceria com a Dialog360. A empresa europeia é autorizada oficialmente pelo WhatsApp a fornecer contas oficiais para uso da API do aplicativo. "
  "O Push WhatsApp Business API, solução desenvolvida pela Ilhasoft, permite utilizar o WhatsApp de forma mais eficiente na comunicação com o seu público. "
  "Com ele, você pode utilizar um único número para integrar o aplicativo a um sistema de atendimento. Assim, você pode direcionar seus clientes automaticamente a atendentes específicos. Ou seja, com um único número, você consegue realizar diversos atendimentos simultâneos. "
  "Além disso, você também pode utilizar este mesmo sistema de atendimento para a comunicação interna da sua empresa. Essa é uma solução para trazer mais produtividade à sua equipe, mesmo em casos de trabalho home office. "
  "Com o Push WhatsApp Business API, também é possível fazer integrações com outros softwares, como ERP’s e CRM’s, e é claro, utilizar chatbots inteligentes. "
  "Por fim, fica claro que a Ilhasoft possui as soluções que sua empresa precisa para otimizar a comunicação com seu público e potencializar estratégias de marketing e vendas. "
  "Afinal, somos uma empresa com vasta experiência em tecnologia e inovação para estratégias de comunicação. Já ganhamos diversos prêmios nacionais e desenvolvemos soluções para clientes renomados mundialmente."
)

while True:
  question = input("Pergunte sobre a Ilhasoft: ")

  query = [
    {
      'context': context,
      'qas': [
          {'id': '0', 'question': question},
      ]
    }
  ]

  pred = model.predict(query)
  print("Pergunta: ", question)
  print("Resposta: ", pred[0][0]['answer'][0])
  print("Confiança: ", pred[1][0]['probability'][0])
  print()

KeyboardInterrupt: Interrupted by user